In [119]:
import os
from dotenv import load_dotenv
import pg8000
import pandas as pd
import datetime as dt
import random as rnd

# Cargar variables de entorno desde .env
load_dotenv()

PASS = os.getenv("POSTGRES_PASSWORD")

# Datos de conexión a la base de datos
db_params = {
    'host': 'localhost',
    'database': 'training',
    'user': 'postgres',
    'password': PASS,
    'port': 5432  # Puerto predeterminado de PostgreSQL
}

In [86]:
# get All table names
try:
    conn = pg8000.connect(**db_params)
    cursor = conn.cursor()
    select = "SELECT table_name FROM information_schema.tables"
    where = "WHERE table_schema = 'public'"
    cursor.execute(f'{select} {where}')
    table_names = cursor.fetchall()

    if table_names:
        table_names = [t[0] for t in table_names]
        print(table_names)
    else:
        print('No tables found')
except pg8000.Error as e:
    print(f'Error pg8000 conn: {e}')

finally:
    if 'cursor' in locals() or conn: 
        cursor.close()
        conn.close()

['empresas', 'vacantes', 'postulantes', 'postulantes_habilidades', 'habilidades', 'postulaciones', 'requerimientos']


In [ ]:
try:
    conn = pg8000.connect(**db_params)
    cursor = conn.cursor()

    dataframes = {}
    # name = 'empresas'
    # cursor.execute(f'select * from {name}')
    # print(cursor.description)
    for name in table_names:
        cursor.execute(f'select * from {name}')
        cols = [item[0] for item in cursor.description]
        rows = cursor.fetchall()
        dataframes[name] = pd.DataFrame(rows, columns=cols)

except pg8000.Error as e:
    print(f'Error while exec query: {e}')
finally:
    if 'cursor' in locals() or conn: 
        cursor.close()
        conn.close()
    

In [88]:
# BASIC QUERIES
empresas = dataframes["empresas"]
vacantes = dataframes["vacantes"]
postulantes = dataframes["postulantes"]
habilidades = ["habilidades"]
postulantes_habilidades = dataframes["postulantes_habilidades"]
postulaciones = dataframes["postulaciones"]
requerimientos = dataframes["requerimientos"]


In [89]:
# empresas del sector salud
# empresas[empresas['sector'] == 'Tecnología']
# vacantes publicadas despues o antes de una fecha dada
# dt.datetime.strptime('2024-1-1',"%Y-%m-%d")
# vacantes['fecha_publicacion']

# vacantes filtradas por fechas (año, mes o fecha completa)
# vacantes['fecha_publicacion'] = pd.to_datetime(vacantes['fecha_publicacion'])
# vacantes_filtradas = vacantes[vacantes['fecha_publicacion'].dt.year < 2024]

# vacantes_filtradas = vacantes[vacantes['fecha_publicacion'].dt.month == 10]

# filter_date = pd.to_datetime('2023-12-1')
# filter_date
# vacantes_filtradas = vacantes[vacantes['fecha_publicacion'] > filter_date]
# vacantes_filtradas

In [103]:
# postulantes que sin email o con correos que no sean gmail
# es importante entender que primero va el filtro de NaN o None
# esto es asi porque si se usa str.contains() antes no va a manejarlos bien y
# va a ignorar el filtro el .isnull()
# postulantes
postulantes[((postulantes['email_postulante'].isnull()) | postulantes['email_postulante'].str.contains("gmail"))]

,id_postulante,nombre_postulante,apellido_postulante,email_postulante
4,4,Pedro,Ruiz,pedro@gmail.com
5,6,Roberto,Santos,None


In [ ]:
# inserts de empresas
# nueva_empresa = {'id_empresa': len(empresas['id_empresa'])+1,'nombre_empresa': 'HTec', 'sector': 'Salud'}
# empresas = pd.concat([empresas, pd.DataFrame([nueva_empresa])], ignore_index=True)

# otra forma de hacerlo es usando loc
# nueva_empresa = {'id_empresa': len(empresas['id_empresa'])+1,'nombre_empresa': 'ITS', 'sector': 'Tecnología'}
# empresas.loc[len(empresas)] = nueva_empresa
# empresas


,id_empresa,nombre_empresa,sector
0,1,Tech Innovators Inc.,Tecnología
1,2,Global Finance Ltd.,Finanzas
2,3,Health Solutions SA,Salud
3,4,HTec,Salud
4,5,ITS,Tecnología


'id_empresa'